## Data Source

https://api.covid19india.org/  
https://api.covid19india.org/csv/

Total Confirmed Cases - https://public.flourish.studio/visualisation/2917845/ <br>
Total Recovered - https://app.flourish.studio/visualisation/2917909/ <br>
Confirmed vs Recovered - https://app.flourish.studio/visualisation/2917932/ <br>
Confirmed vs Deceased - https://app.flourish.studio/visualisation/2917972/ <br>

## Import libraries

In [1]:
import re
import requests
import json
import math
import csv
import numpy as np
import pandas as pd
import folium
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Daily cases

In [49]:
day_wise = pd.read_csv('https://api.covid19india.org/csv/latest/case_time_series.csv')

day_wise.to_csv('/home/adi/Desktop/CODS-NLP-ML-Team/covid-19-india-data/data/nation_level_daily.csv', index=False)

day_wise.tail()

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
139,17 June,13108,367269,6890,194440,341,12263
140,18 June,13829,381098,10741,205181,343,12606
141,19 June,14740,395838,9029,214210,364,12970
142,20 June,15918,411753,13974,228184,308,13278
143,21 June,15158,426911,9069,237253,426,13704


In [8]:
def timeformat(x):
    d = datetime.strptime(x, '%d %B , %Y')
    x = d.strftime('%Y-%m-%d')
    return x

In [9]:
day_wise["Date"] = day_wise["Date"].apply(lambda x: x+", 2020")

In [10]:
day_wise["Date"] = day_wise["Date"].apply(timeformat)

In [11]:
day_wise

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
0,2020-01-30,1,1,0,0,0,0
1,2020-01-31,0,1,0,0,0,0
2,2020-02-01,0,1,0,0,0,0
3,2020-02-02,1,2,0,0,0,0
4,2020-02-03,1,3,0,0,0,0
...,...,...,...,...,...,...,...
139,2020-06-17,13108,367269,6890,194440,341,12263
140,2020-06-18,13829,381098,10741,205181,343,12606
141,2020-06-19,14740,395838,9029,214210,364,12970
142,2020-06-20,15918,411753,13974,228184,308,13278


In [12]:
no_rows = len(day_wise)-1
perspective_deaths = math.floor(day_wise["Daily Deceased"][no_rows]/day_wise["Daily Confirmed"][no_rows]*100)

In [13]:
str(perspective_deaths)+" in 100 people die due to COVID-19 in India based on the current stats."

'2 in 100 people die due to COVID-19 in India based on the current stats.'

## Latest - State wise 

In [70]:
latest = pd.read_csv('https://api.covid19india.org/csv/latest/state_wise.csv')

latest.to_csv('/home/adi/Desktop/CODS-NLP-ML-Team/covid-19-india-data/data/state_level_latest.csv', index=False)
latest.drop(columns=["Migrated_Other","Delta_Confirmed","Delta_Recovered","Delta_Deaths","State_Notes"],axis=1,inplace=True)
latest.drop(latest.index[0],inplace=True)
latest.drop(latest.index[9],inplace=True)
latest.reset_index(drop=True)

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code
0,Maharashtra,132075,65744,6170,60147,21/06/2020 23:04:14,MH
1,Tamil Nadu,59377,32754,757,25866,21/06/2020 18:40:14,TN
2,Delhi,59746,33013,2175,24558,21/06/2020 19:45:14,DL
3,Gujarat,27317,19357,1664,6296,21/06/2020 20:25:15,GJ
4,Uttar Pradesh,17731,10995,550,6186,21/06/2020 20:25:17,UP
5,Rajasthan,14930,11597,349,2984,22/06/2020 00:45:14,RJ
6,West Bengal,13945,8297,555,5093,21/06/2020 20:15:18,WB
7,Madhya Pradesh,11903,9015,515,2373,21/06/2020 21:13:14,MP
8,Haryana,10635,5557,160,4918,21/06/2020 20:15:19,HR
9,Karnataka,9150,5618,138,3390,21/06/2020 20:33:15,KA


In [71]:
df = pd.read_html("https://distancelatlong.com/country/india#table table-striped setBorder")

In [72]:
df = df[2]

In [73]:
latest = latest.sort_values('State')

In [74]:
line = pd.DataFrame({"States": "Ladakh", "Latitude": 34.209515, "Longitude":77.615112}, index=[17])
df2 = pd.concat([df.iloc[:16], line, df.iloc[16:]]).reset_index(drop=True)
line1 = pd.DataFrame({"States": "Gujarat", "Latitude": 22.2587, "Longitude":71.1924}, index=[11])
df3 = pd.concat([df2.iloc[:10], line1, df2.iloc[10:]]).reset_index(drop=True)
line2 = pd.DataFrame({"States": "Telangana", "Latitude": 18.1124, "Longitude":79.0193}, index=[32])
df4 = pd.concat([df3.iloc[:31], line2, df3.iloc[31:]]).reset_index(drop=True)
df4.sort_values("States")
df4.reset_index(drop=True)
df = df4

In [75]:
df = df.sort_values("States")
df.drop(columns="States",axis=1,inplace=True)

In [76]:
latest = latest.reset_index(drop=True)
latest = pd.concat([latest,df],axis=1)

In [77]:
latest

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code,Latitude,Longitude
0,Andaman and Nicobar Islands,48,37,0,11,21/06/2020 19:36:22,AN,11.667026,92.735983
1,Andhra Pradesh,8929,4307,106,4516,21/06/2020 13:36:15,AP,14.750429,78.570026
2,Arunachal Pradesh,139,21,0,118,21/06/2020 22:35:21,AR,27.100399,93.616601
3,Assam,5586,3405,9,2169,22/06/2020 00:13:15,AS,26.749981,94.216667
4,Bihar,7665,5631,51,1983,21/06/2020 21:17:15,BR,25.785414,87.479973
5,Chandigarh,406,316,6,84,21/06/2020 19:55:19,CH,30.719997,76.780006
6,Chhattisgarh,2273,1421,11,841,21/06/2020 21:05:18,CT,22.090420,82.159987
7,Dadra and Nagar Haveli and Daman and Diu,98,26,0,72,21/06/2020 20:46:15,DN,20.266578,73.016618
8,Delhi,59746,33013,2175,24558,21/06/2020 19:45:14,DL,28.669993,77.230004
9,Goa,818,135,0,683,21/06/2020 19:45:19,GA,15.491997,73.818001


In [78]:
latest["Last_Updated_Time"] = latest["Last_Updated_Time"].apply(lambda x:x.split()[0])

In [85]:
tests_state_wise = tests_state_wise.reset_index(drop=True)
latest = latest.astype(str)
latest.to_csv("latest.csv",index=False)

In [80]:
india = folium.Map(location = [20.5937,78.9629],zoom_start=4.5)

for date,state,lat,long,total_cases,Death,Recov,Active in zip(list(latest["Last_Updated_Time"]),list(latest['State']),list(latest['Latitude']),list(latest['Longitude']),list(latest['Confirmed']),list(latest['Deaths']),list(latest['Recovered']),list(latest['Active'])):
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    folium.Marker(location = [lat,long],
                  popup=folium.Popup(('<strong><font color= blue>Date : '+date+'</font></striong><br>' +
                    '<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</striong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</striong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</striong><br>'+
                    '<strong><b>Active Cases : '+Active+'</striong>'  ),max_width=200)).add_to(india)
india

In [81]:
india.save(outfile="india.html")